# Ensemble methods. Exercises


In this section we have only two exercise:

1. Find the best three classifier in the stacking method using the classifiers from scikit-learn package.

2. Build arcing arc-x4 method. 

In [199]:
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

## Exercise 1: Find the best three classifier in the stacking method

Please use the following classifiers:

* Linear regression,
* Nearest Neighbors,
* Linear SVM,
* Decision Tree,
* Naive Bayes,
* QDA.

In [200]:
import numpy as np
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [201]:
def build_classifiers():
    
    LN = LinearRegression()
    KNN = KNeighborsClassifier()
    svc = SVC(kernel='linear', probability=True)
    DT = DecisionTreeClassifier()
    NB = GaussianNB()
    QDA = QuadraticDiscriminantAnalysis()

    classifiers = [LN, KNN, svc, DT, NB, QDA]

    accuracies = []
    for classifier in classifiers:
        classifier.fit(data_set, labels)
        predicted = classifier.predict(data_set)
        if isinstance(classifier, LinearRegression):
            predicted = np.round(predicted)
        acc = accuracy_score(labels, predicted)
        accuracies.append(acc)


    best_three_indices = sorted(range(len(accuracies)), key=lambda i: accuracies[i], reverse=True)[:3]
    best_three = [classifiers[i] for i in best_three_indices]

    return best_three

In [202]:
def build_stacked_classifier(classifiers):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(data_set))
    output = np.array(output).reshape((130,3))
    
    # stacked classifier part:
    stacked_classifier = GaussianNB()
    stacked_classifier.fit(output.reshape((130,3)), labels.reshape((130,)))
    test_set = []
    for classifier in classifiers:
        test_set.append(classifier.predict(test_data_set))
    test_set = np.array(test_set).reshape((len(test_set[0]),3))
    predicted = stacked_classifier.predict(test_set)
    return predicted

In [203]:
classifiers = build_classifiers()
predicted = build_stacked_classifier(classifiers)
accuracy = accuracy_score(test_labels, predicted)
print(accuracy)

1.0


## Exercise 2: 

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [204]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# prepare data set

def generate_data(sample_number, feature_number, label_number):
    data_set = np.random.random_sample((sample_number, feature_number))
    labels = np.random.choice(label_number, sample_number)
    return data_set, labels

labels = 2
dimension = 2
test_set_size = 1000
train_set_size = 5000
train_set, train_labels = generate_data(train_set_size, dimension, labels)
test_set, test_labels = generate_data(test_set_size, dimension, labels)

# init weights
number_of_iterations = 10
weights = np.ones((test_set_size,)) / test_set_size


def train_model(classifier, weights):
    return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)

def calculate_error(model):
    predicted = model.predict(test_set)
    I=calculate_accuracy_vector(predicted, test_labels)
    Z=np.sum(I)
    return (1+Z)/1.0

Fill the two functions below:

In [205]:
def set_new_weights(model):
    weights
    predicted = model.predict(test_set)
    I = (predicted != test_labels).astype(int)
    Z = np.sum(1 + I)
    new_weights = (1 + I) / Z
    return new_weights

Train the classifier with the code below:

In [206]:
classifier = DecisionTreeClassifier(max_depth=1, random_state=1)
classifier.fit(X=train_set, y=train_labels)
alphas = []
classifiers = []
for iteration in range(number_of_iterations):
    model = train_model(classifier, weights)
    weights = set_new_weights(model)
    classifiers.append(model)

print(weights)


validate_x, validate_label = generate_data(1, dimension, labels)

[0.00131406 0.00131406 0.00065703 0.00065703 0.00131406 0.00131406
 0.00065703 0.00065703 0.00131406 0.00065703 0.00065703 0.00131406
 0.00131406 0.00131406 0.00065703 0.00131406 0.00131406 0.00131406
 0.00065703 0.00065703 0.00065703 0.00131406 0.00065703 0.00065703
 0.00131406 0.00065703 0.00131406 0.00065703 0.00065703 0.00065703
 0.00131406 0.00131406 0.00131406 0.00131406 0.00131406 0.00065703
 0.00131406 0.00065703 0.00131406 0.00131406 0.00065703 0.00065703
 0.00065703 0.00065703 0.00065703 0.00065703 0.00065703 0.00065703
 0.00131406 0.00131406 0.00131406 0.00065703 0.00131406 0.00131406
 0.00131406 0.00065703 0.00065703 0.00065703 0.00131406 0.00131406
 0.00131406 0.00131406 0.00131406 0.00065703 0.00065703 0.00131406
 0.00065703 0.00131406 0.00065703 0.00131406 0.00131406 0.00065703
 0.00131406 0.00065703 0.00131406 0.00131406 0.00131406 0.00065703
 0.00065703 0.00131406 0.00065703 0.00065703 0.00131406 0.00131406
 0.00065703 0.00065703 0.00065703 0.00065703 0.00065703 0.0013

Set the validation data set:

In [207]:
validate_x, validate_label = generate_data(1, dimension, labels)

Fill the prediction code:

In [208]:
def get_prediction(x):
    predictions = []
    for model in classifiers:
        predictions.append(model.predict(x)[0])
    counts = np.bincount(predictions)
    return [np.argmax(counts)]

Test it:

In [209]:
prediction = get_prediction(validate_x)[0]

print(f"Przewidziana klasa: {prediction}")
print(f"Prawdziwa klasa: {validate_label[0]}")

if prediction == validate_label[0]:
    print("Predykcja jest poprawna!")
else:
    print("Predykcja jest błędna.")

Przewidziana klasa: 0
Prawdziwa klasa: 0
Predykcja jest poprawna!
